In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2972],
        [0.3830]], grad_fn=<AddmmBackward0>)

In [2]:
# 参数访问
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0503,  0.2038, -0.3490,  0.0383, -0.3210,  0.2705, -0.3284, -0.2974]])),
             ('bias', tensor([0.2913]))])

In [3]:
# 目标参数
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [4]:
print(net[2].bias)

Parameter containing:
tensor([0.2913], requires_grad=True)


In [5]:
print(net[2].bias.data)

tensor([0.2913])


In [6]:
net[2].weight.grad == None

True

In [7]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [10]:
net.state_dict()['2.bias']

tensor([0.2913])

In [11]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.4600,  0.3545,  0.4866, -0.1880],
                      [ 0.1105, -0.4494, -0.3458, -0.3190],
                      [ 0.1584, -0.0858, -0.0562,  0.0209],
                      [-0.1309,  0.4970, -0.2488, -0.2665],
                      [-0.1872, -0.1334,  0.4601,  0.1535],
                      [ 0.4128, -0.2497, -0.0943,  0.4574],
                      [ 0.3421,  0.4655, -0.3339, -0.0393],
                      [ 0.2290,  0.2632, -0.1825, -0.2640]])),
             ('0.bias',
              tensor([-0.0640, -0.4930, -0.1518, -0.4249, -0.2369,  0.1274, -0.4216,  0.0839])),
             ('2.weight',
              tensor([[ 0.0503,  0.2038, -0.3490,  0.0383, -0.3210,  0.2705, -0.3284, -0.2974]])),
             ('2.bias', tensor([0.2913]))])

In [12]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1290],
        [-0.1290]], grad_fn=<AddmmBackward0>)

In [13]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0003,  0.0101,  0.0110, -0.0005]), tensor(0.))

In [21]:
# 使用Xavier初始化方法初始化第一个神经网络层，然后将第三个神经
# 网络层初始化为常量值42
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1470, -0.6419, -0.5429, -0.6775])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
# 自定义初始化
# 有时，深度学习框架没有提供我们需要的初始化方法，在下面的例子中，我们使用以下的分布为任意权重参数w定义初始化方法：

In [35]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[6.1552, -0.0000, 0.0000, 5.7547],
        [0.0000, 7.4708, 9.6807, -0.0000]], grad_fn=<SliceBackward0>)

In [34]:
type(net[2]) == nn.Linear

True

In [32]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [37]:
# 参数绑定
# 我们需要给共享层一个名称，以便可以饮用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [40]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)